In [1]:
import sqlite3
import json
from sqlite3 import Error as Err
         
# connect a database connection to the
# database that resides in the memory
conn = sqlite3.connect("intent_classification.db")
print("Established database connection!")


import json
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
# Load the data from the JSON files
with open("C:\\Users\\atchu\\Downloads\\surprise_data\\surprise_data\\surprise.data", 'r') as data_file:
    data = [json.loads(line) for line in data_file]

with open("C:\\Users\\atchu\\Downloads\\surprise_data\\surprise_data\\surprise.solution", 'r') as solution_file:
    solutions = [json.loads(line) for line in solution_file]


Established database connection!


In [2]:
print(data[0])
print(solutions[0])

{'indoml_id': 'surprise|11109', 'id': '11109', 'utt': 'Can I make a reservation at Buffalo Wild Wings?'}
{'indoml_id': 'surprise|11109', 'intent': 'accept reservations'}


In [3]:
# 1. Merge the data and solutions lists
merged_list = []
for i in range(len(data)):
    merged_item = {
        'indoml_id': data[i]['indoml_id'],
        'utt': data[i]['utt'],
        'intent': solutions[i]['intent']
    }
    merged_list.append(merged_item)
print(merged_list)
print(len(merged_list))
# 2. Organize merged data by intent types
organized_data = {}
for item in merged_list:
    intent = item['intent']
    if intent not in organized_data:
        organized_data[intent] = []
    organized_data[intent].append(item)

print(organized_data)

[{'indoml_id': 'surprise|11109', 'utt': 'Can I make a reservation at Buffalo Wild Wings?', 'intent': 'accept reservations'}, {'indoml_id': 'surprise|11051', 'utt': 'Can I book a table for tonight at Bella Vita?', 'intent': 'accept reservations'}, {'indoml_id': 'surprise|11021', 'utt': 'Does the Cheesecake Factory take reservations?', 'intent': 'accept reservations'}, {'indoml_id': 'surprise|11047', 'utt': 'Do they take reservations at Outback Steakhouse in Las Vegas?', 'intent': 'accept reservations'}, {'indoml_id': 'surprise|11004', 'utt': "does chili's take reservations?", 'intent': 'accept reservations'}, {'indoml_id': 'surprise|11097', 'utt': 'Can I make a reservation at The French Laundry in Yountville?', 'intent': 'accept reservations'}, {'indoml_id': 'surprise|10998', 'utt': 'Can I check availability and make a reservation online?', 'intent': 'accept reservations'}, {'indoml_id': 'surprise|11050', 'utt': "Can you tell me if Carrabba's in Houston accepts reservations", 'intent': 

In [4]:
####Divide it into training and testing data : 80-20 split
merged_train = []
merged_test = []

for intent, items in organized_data.items():
    for i in range(len(items)):
        if i < 12:  # First 12 go to training
            merged_train.append(items[i])
        else:  # The rest (3) go to testing
            
            merged_test.append(items[i])

print(merged_test[0])

{'indoml_id': 'surprise|11105', 'utt': "Where can I find reservation information for Applebee's?", 'intent': 'accept reservations'}


In [5]:
cursor = conn.cursor()
        
ctable = "create table Intent_CL(IDX INT NOT NULL,IndoMLID INT PRIMARY KEY NOT NULL, utt TEXT NOT NULL, intent TEXT NOT NULL);"
cursor.execute(ctable)

In [6]:
for j in merged_train:
    index_intent=str(merged_train.index(j)).replace("'", "''")
    indoml_id = j['indoml_id'].replace("'", "''")
    utt = j['utt'].replace("'", "''")
    intent = j['intent'].replace("'", "''")
    
    inr = f"insert into Intent_CL (IDX,IndoMLID, utt, intent) values ('{index_intent}','{indoml_id}', '{utt}', '{intent}');"
    cursor.execute(inr)

# Commit your changes in the database
conn.commit()


In [7]:

data_l=[]
for i in merged_train:
    str_in=str(i['utt'])+'-'+str(i['intent'])
    data_l.append(str_in)
print(data_l)

['Can I make a reservation at Buffalo Wild Wings?-accept reservations', 'Can I book a table for tonight at Bella Vita?-accept reservations', 'Does the Cheesecake Factory take reservations?-accept reservations', 'Do they take reservations at Outback Steakhouse in Las Vegas?-accept reservations', "does chili's take reservations?-accept reservations", 'Can I make a reservation at The French Laundry in Yountville?-accept reservations', 'Can I check availability and make a reservation online?-accept reservations', "Can you tell me if Carrabba's in Houston accepts reservations-accept reservations", 'Do you have any availability for next weekend at the Brasserie?-accept reservations', 'Does Buffalo Wild Wings take reservations for sports games-accept reservations', 'Is it possible to reserve a table in advance at this restaurant?-accept reservations', 'Does Olive Garden in San Francisco take reservations?-accept reservations', 'Why did my brokerage account suddenly get locked?-account blocked

In [8]:
print(merged_train[0])

{'indoml_id': 'surprise|11109', 'utt': 'Can I make a reservation at Buffalo Wild Wings?', 'intent': 'accept reservations'}


In [9]:
# subset_data=data_l[0:10]
from txtai.embeddings import Embeddings

# Create embeddings model, backed by sentence-transformers & transformers
embeddings = Embeddings(path="sentence-transformers/nli-mpnet-base-v2")

# Index the list of text
embeddings.index(data_l)


print(f"{'Query':20} Best Match")
print("-" * 50)

query = 'Reserve a table for me'
uid = embeddings.search(query,0)[0][0]


# Print text
print(f"{query:20} {data_l[uid]}")
print("Answer is")
index_intent=data_l.index(data_l[uid])
print(index_intent)

Query                Best Match
--------------------------------------------------
Reserve a table for me Is it possible to reserve a table in advance at this restaurant?-accept reservations
Answer is
10


In [10]:
print(merged_train[index_intent]['intent'])

accept reservations


In [11]:
test_query=[];test_int=[]
for i in merged_test:
    test_query.append(i['utt'])
    test_int.append(i['intent'])


predicted_int=[] ### predicted intent
for q in range(0,len(test_query)):
    query=test_query[q]
    uid = embeddings.search(query,0)[0][0]
    index_intent=data_l.index(data_l[uid])
    predicted_int.append(merged_train[index_intent]['intent'])
print(predicted_int)  

['accept reservations', 'book hotel', 'confirm reservation', 'account blocked', 'account blocked', 'account blocked', 'alarm', 'alarm', 'alarm', 'application status', 'application status', 'application status', 'apr', 'apr', 'apr', 'are you a bot', 'are you a bot', 'are you a bot', 'todo list update', 'balance', 'balance', 'bill balance', 'balance', 'insurance', 'bill due', 'bill due', 'bill due', 'book flight', 'book flight', 'book flight', 'book hotel', 'directions', 'book hotel', 'calculator', 'calculator', 'mpg', 'calendar update', 'todo list update', 'calendar', 'schedule meeting', 'calendar update', 'calendar update', 'calories', 'calories', 'calories', 'cancel', 'cancel', 'cancel', 'cancel reservation', 'cancel reservation', 'confirm reservation', 'car rental', 'car rental', 'car rental', 'card declined', 'card declined', 'card declined', 'carry on', 'carry on', 'carry on', 'change language', 'change accent', 'change volume', 'change ai name', 'change ai name', 'change ai name',

In [12]:

report = classification_report(test_int, predicted_int, output_dict=True)

accuracy = report['accuracy']
precision = report['macro avg']['precision']
recall = report['macro avg']['recall']
f1_score = report['macro avg']['f1-score']

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

Accuracy: 0.8036
Precision: 0.8408
Recall: 0.8044
F1 Score: 0.7998
